<a href="https://colab.research.google.com/github/ryanbodrug/EMNIST/blob/master/emnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2019 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow 2 quickstart for beginners

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/quickstart/beginner"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This short introduction uses [Keras](https://www.tensorflow.org/guide/keras/overview) to:

1. Build a neural network that classifies images.
2. Train this neural network.
3. And, finally, evaluate the accuracy of the model.

This is a [Google Colaboratory](https://colab.research.google.com/notebooks/welcome.ipynb) notebook file. Python programs are run directly in the browser—a great way to learn and use TensorFlow. To follow this tutorial, run the notebook in Google Colab by clicking the button at the top of this page.

1. In Colab, connect to a Python runtime: At the top-right of the menu bar, select *CONNECT*.
2. Run all the notebook code cells: Select *Runtime* > *Run all*.

Download and install TensorFlow 2. Import TensorFlow into your program:

Note: Upgrade `pip` to install the TensorFlow 2 package. See the [install guide](https://www.tensorflow.org/install) for details.

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import matplotlib.pyplot as plt
import numpy as np
import tensorflow.compat.v2 as tf
#import tensorflow_datasets.public_api as tfds

import tensorflow_datasets as tfds
tf.enable_v2_behavior()

Load and prepare the [EMNIST dataset](https://www.nist.gov/itl/products-and-services/emnist-dataset/). 

In [0]:
# load train data
print("loading emnist train data")
emnist_data, emnist_info = tfds.load(name="emnist/balanced:3.*.*", with_info=True, as_supervised=True)
print(emnist_data)
print(emnist_info)




Setup image preprocessing: 

In [0]:
# Scaling EMNIST data from (0, 255] to (0., 1.]
# Transpose to make format more human friendly.  See note Note: Like the original EMNIST data, images provided here are inverted horizontally and rotated 90 anti-clockwise. You can use tf.transpose within ds.map to convert the images to a human-friendlier format.
BATCH_SIZE = 128
def transform(image, label):
    image = tf.transpose(image)
    image = tf.cast(image, tf.float32)
    image /= 255
    return image, label

emnist_train = emnist_data["train"].map(transform).batch(BATCH_SIZE)
emnist_test = emnist_data["test"].map(transform).batch(BATCH_SIZE)


Create a mapping of label ints to characters. 

In [0]:
def create_label_map():
  map = {}
  for i in range(62):
    if(i >= 0 and i <= 9):
      map[i] = chr(i + 48)
    elif(i >= 10 and i <= 35):
      map[i] = chr(i + 55)
    elif(i >= 36 and i <= 61):
      map[i] = chr(i + 61)
  return map

label_map = create_label_map()
print(label_map)

Visualize a sample of the test and train data

In [0]:
# This function will plot images in the form of a grid with 1 row and 5 columns where images are placed in each column.
def plot_images(data, size):
    images_arr = [x[0] for x in data]
    labels_arr = [label_map[x[1].numpy()] for x in data]
    fig, axes = plt.subplots(1, size, figsize=(10,10))
    axes = axes.flatten()
    for img, label, ax in zip( images_arr, labels_arr, axes):
        ax.imshow(img.numpy().squeeze(), cmap='gray')
        ax.axis('off')
        ax.set_title(label)
    plt.tight_layout()
    plt.show()

print("Sample Train Images")
unbatch_train = emnist_train.unbatch().take(5)
plot_images(unbatch_train, 5)

print("Sample Test Images")
unbatch_test = emnist_test.unbatch().take(5)
plot_images(unbatch_test, 5)


Build the `tf.keras.Sequential` model by stacking layers. Choose an optimizer and loss function for training:

In [0]:

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(emnist_info.features['label'].num_classes)
])

For each example the model returns a vector of "[logits](https://developers.google.com/machine-learning/glossary#logits)" or "[log-odds](https://developers.google.com/machine-learning/glossary#log-odds)" scores, one for each class.

In [0]:


example, = emnist_train.take(1)
image = example[0]
label = example[1]
labels = [label_map[l] for l in label.numpy()]

print("Predictions for: ", labels)
predictions = model(image).numpy()
predictions




The `tf.nn.softmax` function converts these logits to "probabilities" for each class: 

In [0]:
tf.nn.softmax(predictions).numpy()

Note: It is possible to bake this `tf.nn.softmax` in as the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to
provide an exact and numerically stable loss calculation for all models when using a softmax output. 

The `losses.SparseCategoricalCrossentropy` loss takes a vector of logits and a `True` index and returns a scalar loss for each example.

In [0]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This loss is equal to the negative log probability of the the true class:
It is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/47 for each class), so the initial loss should be close to `-tf.log(1/47) ~= 3.85`.

In [0]:
-tf.math.log(1/47)

In [0]:
loss_fn(label, predictions).numpy()

In [0]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

The `Model.fit` method adjusts the model parameters to minimize the loss: 

In [0]:
model.fit(emnist_train, epochs=5)

The `Model.evaluate` method checks the models performance, usually on a "[Validation-set](https://developers.google.com/machine-learning/glossary#validation-set)".

In [0]:
model.evaluate(emnist_test, verbose=2)

The image classifier is now trained to ~81% accuracy on this dataset. To learn more, read the [TensorFlow tutorials](https://www.tensorflow.org/tutorials/).

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [0]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [0]:
# x[0] is the tuple version of x["images"]
train_images = [x[0] for x in emnist_train.take(5)]
probability_model(train_images)